In [1]:
import copy
import numpy as np

### Define functions

- Sigmoid Function $$ \sigma(x) = {1 \over 1 + e^{-x}} $$
- Sigmoid Derivative $$ \sigma'(x) = \sigma(x) \cdot (1-\sigma(x)) $$

In [2]:
# Define sigmoid function
def sigmoid(x):
    output = 1/(1 + np.exp(-x))
    return output

# Define sigmoid derivative
def sigmoid_derivative(sigmoid_x):
    return sigmoid_x * (1 - sigmoid_x)

In [3]:
# Generate training dataset
int2binary = {}
binary_dim = 8

largest_number = pow(2,binary_dim)
binary = np.unpackbits(
    np.array([range(largest_number)],dtype=np.uint8).T, axis=1)

for i in range(largest_number):
    int2binary[i] = binary[i]

In [4]:
int2binary

{0: array([0, 0, 0, 0, 0, 0, 0, 0], dtype=uint8),
 1: array([0, 0, 0, 0, 0, 0, 0, 1], dtype=uint8),
 2: array([0, 0, 0, 0, 0, 0, 1, 0], dtype=uint8),
 3: array([0, 0, 0, 0, 0, 0, 1, 1], dtype=uint8),
 4: array([0, 0, 0, 0, 0, 1, 0, 0], dtype=uint8),
 5: array([0, 0, 0, 0, 0, 1, 0, 1], dtype=uint8),
 6: array([0, 0, 0, 0, 0, 1, 1, 0], dtype=uint8),
 7: array([0, 0, 0, 0, 0, 1, 1, 1], dtype=uint8),
 8: array([0, 0, 0, 0, 1, 0, 0, 0], dtype=uint8),
 9: array([0, 0, 0, 0, 1, 0, 0, 1], dtype=uint8),
 10: array([0, 0, 0, 0, 1, 0, 1, 0], dtype=uint8),
 11: array([0, 0, 0, 0, 1, 0, 1, 1], dtype=uint8),
 12: array([0, 0, 0, 0, 1, 1, 0, 0], dtype=uint8),
 13: array([0, 0, 0, 0, 1, 1, 0, 1], dtype=uint8),
 14: array([0, 0, 0, 0, 1, 1, 1, 0], dtype=uint8),
 15: array([0, 0, 0, 0, 1, 1, 1, 1], dtype=uint8),
 16: array([0, 0, 0, 1, 0, 0, 0, 0], dtype=uint8),
 17: array([0, 0, 0, 1, 0, 0, 0, 1], dtype=uint8),
 18: array([0, 0, 0, 1, 0, 0, 1, 0], dtype=uint8),
 19: array([0, 0, 0, 1, 0, 0, 1, 1], dtyp

In [5]:
# Input variables
learning_rate = 0.3
input_dim = 2
hidden_dim = 16
output_dim = 1

In [6]:
# Initialize weights
wih = 2 * np.random.random((input_dim, hidden_dim)) - 1
who = 2 * np.random.random((hidden_dim, output_dim)) - 1
whh = 2 * np.random.random((hidden_dim, hidden_dim)) - 1

update_wih = np.zeros_like(wih)
update_who = np.zeros_like(who)
update_whh = np.zeros_like(whh)

In [11]:
# Training
iteration = 10000

for iterate in range(iteration):
    # generate a simple addition problem (a + b = c)
    a_int = np.random.randint(largest_number/2) # int version
    a = int2binary[a_int] # binary encoding

    b_int = np.random.randint(largest_number/2) # int version
    b = int2binary[b_int] # binary encoding
    
    # True answer
    c_int = a_int + b_int
    c = int2binary[c_int]

    # where we'll store our best guess (binary encoded)
    d = np.zeros_like(c)
    overallError = 0

    output_deltas = list()
    hidden_values = list()
    hidden_values.append(np.zeros(hidden_dim))
    
    # Moving along the positions in the binary encoding
    for position in range(binary_dim):
        # Generate input and output
        X = np.array([
            [a[binary_dim - position - 1],
             b[binary_dim - position - 1]]])

        y = np.array([[c[binary_dim - position - 1]]]).T

        # Hidden layer (input ~+ prev_hidden)
        hidden_layer = sigmoid(np.dot(X, wih) + np.dot(hidden_values[-1], whh))
#        print(hidden_layer)
        # output layer (new binary representation)
        output_layer = sigmoid(np.dot(hidden_layer,who))

        # did we miss?... if so, by how much?
        output_error = y - output_layer
        output_deltas.append((output_error) * sigmoid_derivative(output_layer))
        overallError += np.abs(output_error[0])
    
        # decode estimate so we can print it out
        d[binary_dim - position - 1] = np.round(output_layer[0][0])
#        print(output_layer)
        # store hidden layer so we can use it in the next timestep
        hidden_values.append(copy.deepcopy(hidden_layer))
        
    future_hidden_delta = np.zeros(hidden_dim)
    
    for position in range(binary_dim):
        X = np.array([[a[position], b[position]]])
        hidden_layer = hidden_values[-position -1]
        prev_hidden = hidden_values[-position -2]
#        print(len(hidden_values), hidden_values)
        # error at output layer
        output_delta = output_deltas[-position -1]
        # error at hidden layer
        hidden_delta = (future_hidden_delta.dot(whh.T)
                        + output_delta.dot(who.T)) * sigmoid_derivative(hidden_layer)

        # let's update all our weights so we can try again
        update_who += np.atleast_2d(hidden_layer).T.dot(output_delta)
        update_whh += np.atleast_2d(prev_hidden).T.dot(hidden_delta)
        update_wih += X.T.dot(hidden_delta)
        
        future_hidden_delta = hidden_delta

    wih += update_wih * learning_rate
    who += update_who * learning_rate
    whh += update_whh * learning_rate

    update_wih = 0
    update_who = 0
    update_whh = 0
    # print out progress
    if iterate % (iteration/10) == 0:
        print("Iteration:", iterate)
        print("Error:" + str(overallError))
        print("Pred:" + str(d))
        print("True:" + str(c))
        out = 0
        for index,x in enumerate(reversed(d)):
            out += x*pow(2,index)
        print(str(a_int) + " + " + str(b_int) + " = " + str(out))
        print("------------")

[[0]] [[ 0.715086]] [[-0.715086]]
[array([[-0.1456902]])]
[[1]] [[ 0.588029]] [[ 0.411971]]
[array([[-0.1456902]]), array([[ 0.09980034]])]
[[0]] [[ 0.46491614]] [[-0.46491614]]
[array([[-0.1456902]]), array([[ 0.09980034]]), array([[-0.11565678]])]
[[1]] [[ 0.69802908]] [[ 0.30197092]]
[array([[-0.1456902]]), array([[ 0.09980034]]), array([[-0.11565678]]), array([[ 0.06365079]])]
[[1]] [[ 0.52683458]] [[ 0.47316542]]
[array([[-0.1456902]]), array([[ 0.09980034]]), array([[-0.11565678]]), array([[ 0.06365079]]), array([[ 0.11795063]])]
[[0]] [[ 0.50468882]] [[-0.50468882]]
[array([[-0.1456902]]), array([[ 0.09980034]]), array([[-0.11565678]]), array([[ 0.06365079]]), array([[ 0.11795063]]), array([[-0.12616111]])]
[[0]] [[ 0.61721187]] [[-0.61721187]]
[array([[-0.1456902]]), array([[ 0.09980034]]), array([[-0.11565678]]), array([[ 0.06365079]]), array([[ 0.11795063]]), array([[-0.12616111]]), array([[-0.14582333]])]
[[1]] [[ 0.67018643]] [[ 0.32981357]]
[array([[-0.1456902]]), array([[